# Homework 1

Due on: 5/30.  Please upload your completed assignment to Canvas

1. For the following words: logistic, logistics, shoe, shoes

   a. Porter stem with nltk
   
   b. lemmatize with nltk
   
   c. lemmatize with Spacy

In [32]:
import nltk
import spacy
nlp = spacy.load("en_core_web_sm")
import pandas as pd
from scipy import linalg

In [40]:
import matplotlib.pyplot as plt
%matplotlib_inline

UsageError: Line magic function `%matplotlib_inline` not found.


In [67]:
from sklearn import decomposition
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [5]:
word_list = ['logistic','logistics','shoe','shoes']

porter = nltk.stem.porter.PorterStemmer()
print([porter.stem(w) for w in word_list])

lemmatizer = nltk.stem.WordNetLemmatizer()
print([lemmatizer.lemmatize(w) for w in word_list])

# lookups = spacy.lookups.Lookups()
# splemmatizer = spacy.lemmatizer.Lemmatizer(lookups)
# splemmatizer('hello')
# print([splemmatizer(w) for w in word_list])

from spacy.lemmatizer import Lemmatizer
from spacy.lookups import Lookups
lookups = Lookups()
# lookups.add_table("lemma_rules", {"noun": [["s", ""]]})

lemmatizer = Lemmatizer(lookups)
[lemmatizer(w, 1) for w in word_list]

['logist', 'logist', 'shoe', 'shoe']
['logistic', 'logistics', 'shoe', 'shoe']


[['logistic'], ['logistics'], ['shoe'], ['shoes']]

2\. n-grams are an important NLP concept.  An n-gram is a contiguous sequence of n items (where the items can be characters, syllables, or words).  Here, we  A 1-gram is a unigram, a 2-gram is a bigram, and a 3-gram is a trigram.

Here, we are referring to sequences of words. The sentence "It was a bright cold day in April." contains the following trigrams:

- It was a
- was a bright
- a bright cold
- bright cold day
- cold day in
- day in April

Write a function that returns a dictionary with the n-grams of a text (for `min_n <= n <= max_n`) and a count of how often they appear:

In [6]:
def get_ngrams(text, min_n, max_n):
    
    #Exercise: FILL IN METHOD
    doc = nlp(text)
    words = [str(d) for d in doc if not d.is_punct]
    ngram_dict = {}
    
    for n in range(min_n, max_n + 1):
        for w in range(len(words)-n+1):
            ngram = " ".join(words[w:w+n])
            if ngram in ngram_dict.keys():
                ngram_dict[ngram] += 1
            else:
                ngram_dict[ngram] = 1
            
    return ngram_dict

text = "I like fish, and I like chicken and I like fish."
get_ngrams(text, 2, 3)

{'I like': 3,
 'like fish': 2,
 'fish and': 1,
 'and I': 2,
 'like chicken': 1,
 'chicken and': 1,
 'I like fish': 2,
 'like fish and': 1,
 'fish and I': 1,
 'and I like': 2,
 'I like chicken': 1,
 'like chicken and': 1,
 'chicken and I': 1}

3\. Write a method that given a list of strings (you can think of each string as a document), returns a dictionary for each string, where the keys are the vocabulary words, and the values are the frequencies.

In [7]:
def get_vocab_frequency(list_of_strings):
    ngram_dicts = []
    for text in list_of_strings:
        ngram_dict = get_ngrams(text, 1, 1)
        ngram_dicts.append(ngram_dict)
    return ngram_dicts

get_vocab_frequency(["hello! hello? how are you? are you ok?","la la la la","do ray me fa so la ti doh"])

[{'hello': 2, 'how': 1, 'are': 2, 'you': 2, 'ok': 1},
 {'la': 4},
 {'do': 1, 'ray': 1, 'me': 1, 'fa': 1, 'so': 1, 'la': 1, 'ti': 1, 'doh': 1}]

4\. Write a method that when given a list of strings (you can think of each string as a document), calculates the TF-IDF, and returns a term-document matrix with the results. It will be useful to use your `get_vocab_frequency` method from problem 3.

In [8]:
lst = ['a','b','a']
lst.count('a')

2

In [9]:
import numpy as np
np.log(0.4)

-0.916290731874155

In [10]:
# TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
# IDF(t) = log_e(Total number of documents / Number of documents with term t in it)
def get_tfidf(list_of_strings):
    vocab = get_vocab_frequency(list_of_strings)
    all_words = []
    for v in vocab:
        all_words.extend(v.keys())
    all_words = list(set(all_words))
    matrix = {word:[] for word in all_words}

    for i, v in enumerate(vocab):
        word_count = sum([freq for key,freq in v.items()])
        doc_tf = {word:(freq/word_count) for word,freq in v.items()}
        doc_idf = {word:np.log(len(vocab)/len([doc for doc in vocab if word in doc.keys()])) for word,freq in v.items()}
        doc_tfidf = {word:tf * doc_idf[word] for word, tf in doc_tf.items()}
        
        for word in matrix.keys():
            if word in doc_tfidf.keys():
                matrix[word].append(doc_tfidf[word])
            else:
                matrix[word].append(0)
    df_matrix = pd.DataFrame(matrix)
    return df_matrix

get_tfidf(["hello! hello? how are you? are you ok? how is your cat?","my cat is the best!","do ray me fa so la ti doh"])


,hello,your,how,my,ray,are,la,fa,you,cat,ok,do,so,best,the,ti,is,doh,me
0,0.183102,0.091551,0.183102,0.000000,0.000000,0.183102,0.000000,0.000000,0.183102,0.033789,0.091551,0.000000,0.000000,0.000000,0.000000,0.000000,0.033789,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.219722,0.000000,0.000000,0.000000,0.000000,0.000000,0.081093,0.000000,0.000000,0.000000,0.219722,0.219722,0.000000,0.081093,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.137327,0.000000,0.137327,0.137327,0.000000,0.000000,0.000000,0.137327,0.137327,0.000000,0.000000,0.137327,0.000000,0.137327,0.137327


5\. Who said the following (*Hint: Be sure to read the class notebooks and relevant links*):

A. "It's true there's been a lot of work on trying to apply statistical models to various linguistic problems. I think there have been some successes, but a lot of failures. There is a notion of success ... which I think is novel in the history of science. It interprets success as approximating unanalyzed data."
chomsky

B. "I agree that it can be difficult to make sense of a model containing billions of parameters. Certainly a human can't understand such a model by inspecting the values of each parameter individually. But one can gain insight by examing the properties of the model—where it succeeds and fails, how well it learns as a function of data, etc."
peter norvig

norvig also said:
Einstein said to make everything as simple as possible, but no simpler. Many phenomena in science are stochastic, and the simplest model of them is a probabilistic model; I believe language is such a phenomenon and therefore that probabilistic models are our best tool for representing facts about language, for algorithmically processing language, and for understanding how humans process language.

C. The big-data big-compute paradigm of modern Deep Learning has in fact “perverted the field” (of computational linguistics) and “sent it off-track”
chris manning

D. Language is crucial to general intelligence, because language is the conduit by which individual intelligence is shared and transformed into societal intelligence.
chris manning

E. Structure is a “necessary evil”, and warned that imposing structure requires us to make certain assumptions, which are invariably wrong for at least some portion of the data, and may become obsolete within the near future.
lecun

In [72]:
def get_term_doc_test_train_sets(filepath, train_set_size, col = 'Name',):
    df = pd.read_csv(filepath)
    df = df[[col,"Text"]]
    target_names = df.Name.unique().tolist()
    df["target"] = df.apply(lambda x: target_names.index(x.Name), axis = 1)
    df = df.rename(columns = {'Text':'data'})
    df = df[["target","data"]]
    shuffled = df.sample(frac =1)
    train_set = {'data':shuffled[0:train_set_size].data.tolist(), 'target_names':target_names, 'target':shuffled[0:train_set_size].target.tolist()}
    test_set = {'data':shuffled[train_set_size:].data.tolist(), 'target_names':target_names, 'target':shuffled[train_set_size:].target.tolist()}
    return train_set, test_set

rtrain, rtest = get_term_doc_test_train_sets("/home/tessa/reading_age/projects/reddit_gen_spef/1_chunk/scored.csv", col = 'Name', train_set_size = 4000)

In [77]:
def vectorize(term_doc, how = 'count'):
    if how == 'count':
        vectorizer = CountVectorizer(stop_words='english')
    elif how == 'tfidf':
        vectorizer = TfidfVectorizer(stop_words = 'english')
    vectors = vectorizer.fit_transform(term_doc['data']).todense()
    vocab = np.array(vectorizer.get_feature_names())
    return vectors, vocab

In [80]:
def topics_from_svd(vectors, vocab, num_top_words=10):
    U, s, Vh = linalg.svd(vectors, full_matrices=False)
    topics = show_topics(Vh, vocab, num_top_words=10)
    return topics
# rvectors, rvocab = vectorize(rtrain, how = 'tfidf')
# topics_from_svd(rvectors, rvocab)

In [82]:
def topics_from_nmf(vectors, vocab, num_topics, num_top_words = 10):
    m,n=vectors.shape
    d=num_topics
    clf = decomposition.NMF(n_components=d, random_state=1)

    W1 = clf.fit_transform(vectors)
    H1 = clf.components_
    topics = show_topics(H1, vocab, num_top_words = num_top_words)
    return topics
rvectors, rvocab = vectorize(rtrain, how = 'tfidf')
topics_from_nmf(rvectors, rvocab, 4, num_top_words = 20)

['like just people don really think know ve thing say good going actually way didn lot time feel drag did',
 'http com www https org amp wiki wikipedia reddit en youtube comments watch jpg imgur html link edit article google',
 'game dm players play amp character group new playing time want make campaign player rules fun friends set ve need',
 'god gt believe jesus christ bible faith christian church sin life christians pray love word scripture don things lord says']

In [83]:
rvectors, rvocab = vectorize(rtrain, how = 'tfidf')
topics_from_nmf(rvectors, rvocab, 10)

['just really ve time good thing did didn actually think',
 'com www http https reddit comments youtube watch imgur jpg',
 'game dm players play group character new campaign rules player',
 'god love jesus believe sin pray word life faith lord',
 'wiki org wikipedia http en article html www english page',
 'people don know think want need ll things way say',
 'amp google https hl utf playing books source com new',
 'church christian christ believe bible christians jesus faith catholic christianity',
 'like feel drag cracker trixie looks look queens aquaria queen',
 'gt men man gods long lust women assume understand saying']

In [84]:
rvectors, rvocab = vectorize(rtrain, how = 'tfidf')
topics_from_nmf(rvectors, rvocab, 10, num_top_words = 20)

['just really ve time good thing did didn actually think got going thought bad said way mean use say doesn',
 'com www http https reddit comments youtube watch imgur jpg edit link html todayilearned video rupaulsdragrace autplayed ignoreme imguralbumbot thread',
 'game dm players play group character new campaign rules player make playing want time fun friends set characters help need',
 'god love jesus believe sin pray word life faith lord bible things man christ father holy grace doesn scripture plan',
 'wiki org wikipedia http en article html www english page crazy jpg ones different french uk nuclear billion number cot',
 'people don know think want need ll things way say really understand wrong make hate care drag does fact problem',
 'amp google https hl utf playing books source com new sa played search looking imguralbumbot autplayed ignoreme biw games bih',
 'church christian christ believe bible christians jesus faith catholic christianity years say scripture says gospel sayin

In [11]:
# topic modelling - try and divide some texts into topics
# first get a dataframe of different texts

novels = pd.read_csv("/home/tessa/reading_age/projects/24_novels/chunks/scored.csv")
novels = novels[["Name","Text"]]
novels

,Name,Text
0,Peter_Pan,PREFACE Sir J. M. Barrie's delightful creation...
1,Peter_Pan,Then she turned the bedclothes neatly down and...
2,Peter_Pan,Darling told him about the weird apparition at...
3,Peter_Pan,"Suddenly the night-lights flickered, waned, an..."
4,Peter_Pan,No wonder he was crying! But that was not the ...
...,...,...
5271,Three_men_in_a_boat-Jerome_K,"George took the fun more soberly, and stuck to..."
5272,Three_men_in_a_boat-Jerome_K,Harris said he would introduce us both to the ...
5273,Three_men_in_a_boat-Jerome_K,"After which, we managed to get some fitful slu..."
5274,Three_men_in_a_boat-Jerome_K,“Yes it’s almost a pity we’ve made up our mind...


In [12]:
target_names = novels.Name.unique().tolist()
target_names

['Peter_Pan',
 'Ulysses',
 'A_Study_in_Scarlet',
 'A_Midsummer_Nights_Dream',
 'Alice_in_Wonderland',
 'Dracula',
 'Five_Famous_Fairy_Tales',
 'Gullivers_Travels',
 'Les_Miserables',
 'Nicholas_Nickleby',
 'Pride_And_Prejudice',
 'Romeo_and_Juliet',
 'Sleeping_Beauty',
 'The_Disappearance_of_Lady_Frances_Carfax',
 'The_Wind_in_the_Willows',
 'The_Tragical_History_of_Doctor_Faustus',
 'Three_men_in_a_boat-Jerome_K']

In [13]:
novels["target"] = novels.apply(lambda x: target_names.index(x.Name), axis = 1)
novels

,Name,Text,target
0,Peter_Pan,PREFACE Sir J. M. Barrie's delightful creation...,0
1,Peter_Pan,Then she turned the bedclothes neatly down and...,0
2,Peter_Pan,Darling told him about the weird apparition at...,0
3,Peter_Pan,"Suddenly the night-lights flickered, waned, an...",0
4,Peter_Pan,No wonder he was crying! But that was not the ...,0
...,...,...,...
5271,Three_men_in_a_boat-Jerome_K,"George took the fun more soberly, and stuck to...",16
5272,Three_men_in_a_boat-Jerome_K,Harris said he would introduce us both to the ...,16
5273,Three_men_in_a_boat-Jerome_K,"After which, we managed to get some fitful slu...",16
5274,Three_men_in_a_boat-Jerome_K,“Yes it’s almost a pity we’ve made up our mind...,16


In [14]:
novels = novels.rename(columns = {'Text':'data'})
novels = novels[["target","data"]]

In [15]:
shuffled = novels.sample(frac =1)

In [16]:
train_set = {'data':shuffled[0:4000].data.tolist(), 'target_names':target_names, 'target':shuffled[0:4000].target.tolist()}
test_set = {'data':shuffled[4000:].data.tolist(), 'target_names':target_names, 'target':shuffled[4000:].target.tolist()}

In [17]:
get_tfidf(train_set['data'][0:3])

,contained,Oxford,descends,place,occurred,talk,high,out,need,fallen,...,immense,obligations,be,few,below,not,you,called,respects,several
0,0.000000,0.000000,0.002795,0.002063,0.000000,0.005591,0.005591,0.003095,0.000000,0.002795,...,0.002795,0.000000,0.000000,0.000000,0.002795,0.000000,0.003095,0.002795,0.000000,0.000000
1,0.000000,0.003231,0.000000,0.000000,0.003231,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.003231,0.003578,0.000000,0.000000,0.000000,0.001193,0.000000,0.000000,0.006462
2,0.002876,0.000000,0.000000,0.001061,0.000000,0.000000,0.000000,0.001061,0.002876,0.000000,...,0.000000,0.000000,0.002123,0.002876,0.000000,0.005752,0.000000,0.000000,0.002876,0.000000


In [19]:

vectorizer = CountVectorizer(stop_words='english')

In [20]:
# so this just gives a count matrix of the number of times each word appears (ignoring stop words)
vectors = vectorizer.fit_transform(train_set['data']).todense()

In [22]:
vocab = np.array(vectorizer.get_feature_names())

In [33]:
# now factorizing the matrix into 3 other matricies
%time U, s, Vh = linalg.svd(vectors, full_matrices=False)

CPU times: user 17min 38s, sys: 23min 47s, total: 41min 25s
Wall time: 2min 51s


In [55]:
# lots of references to valjean, and nicholas - seems to be overly weighted by the longest books 
def show_topics(a, vocab, num_top_words=10):
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

show_topics(Vh[10:20], vocab)

['miss squeers nicholas says man marius know did father john',
 'miss bloom man mantalini squeers door la old nickleby madame',
 'man jean faustus valjean good squeers know javert toad shall',
 'marius faustus sir squeers shall cosette know come soul thou',
 'squeers ralph romeo love mrs boys don boy little juliet',
 'ralph mr know old mantalini newman nickleby don love arthur',
 'know sir like bloom toad time miss did ll don',
 'ralph old door house toad marius says room elizabeth rue',
 'ralph little says newman mantalini miss eyes did cosette sir',
 'sir old good miss mulberry young say lady day shall']

In [44]:
# m is equal to r in my SVD and the one in the example - why?
tf_vectorizer = TfidfVectorizer(stop_words = 'english')

In [56]:
tf_vocab = np.array(tf_vectorizer.get_feature_names())

In [46]:
tf_vectors = tf_vectorizer.fit_transform(train_set['data']).todense()

In [47]:
tf_vectors.shape

(4000, 41889)

In [51]:
np.max(tf_vectors[2])

0.46616561253062055

In [52]:
%time tU, ts, tVh = linalg.svd(tf_vectors, full_matrices=False)

CPU times: user 18min, sys: 24min 52s, total: 42min 53s
Wall time: 3min 51s


In [58]:
print(tU.shape, ts.shape, tVh.shape)

(4000, 4000) (4000,) (4000, 41889)


In [60]:
show_topics(tVh[0:10], tf_vocab)

['autem irving leopoldi kossuth eius breastplate legplates tyler generatio vocabitur',
 'nicholas mr squeers mrs miss nickleby elizabeth ralph kate darcy',
 'elizabeth darcy jane bennet bingley mr collins wickham lydia catherine',
 'jean valjean cosette marius nicholas squeers javert nickleby mr miss',
 'toad rat mole van helsing alice lucy mina went badger',
 'ralph nickleby kate mantalini mulberry newman arthur mrs wititterly helsing',
 'squeers helsing van lucy mina jonathan arthur dracula valjean professor',
 'valjean jean toad rat mole javert badger alice ll cosette',
 'nicholas crummles newman tim smike linkinwater manager brother creevy la',
 'marius cosette toad rat mole nicholas badger thénardier know gillenormand']

In [63]:
# this looks way better than the svd -> i've only done this with 5 topics but would be interesting to expand
m,n=vectors.shape
d=5
clf = decomposition.NMF(n_components=d, random_state=1)

W1 = clf.fit_transform(vectors)
H1 = clf.components_
show_topics(H1, vocab)

['said know don alice asked ll went like yes come',
 'man little did like old time marius jean valjean day',
 'mr elizabeth mrs darcy bennet miss jane bingley lady collins',
 'thou romeo love thy thee faustus shall juliet come ll',
 'nicholas squeers miss nickleby sir mrs said ralph kate replied']

In [64]:
show_topics(H1, vocab)

['said know don alice asked ll went like yes come',
 'man little did like old time marius jean valjean day',
 'mr elizabeth mrs darcy bennet miss jane bingley lady collins',
 'thou romeo love thy thee faustus shall juliet come ll',
 'nicholas squeers miss nickleby sir mrs said ralph kate replied']

In [65]:
tm,tn=vectors.shape
td=10
clf = decomposition.NMF(n_components=td, random_state=1)

tW1 = clf.fit_transform(tf_vectors)
tH1 = clf.components_
show_topics(tH1, tf_vocab)

['said little man like old alice time did bloom great',
 'valjean jean cosette javert fauchelevent madeleine rue man convict did',
 'elizabeth mr darcy bennet jane bingley mrs miss collins wickham',
 'ralph nickleby kate mrs said mantalini sir mulberry mr miss',
 'romeo thou juliet thy thee faustus love capulet friar nurse',
 'squeers miss boys mr said nicholas boy replied tilda smike',
 'helsing van lucy mina jonathan arthur shall professor dracula count',
 'toad rat mole badger ll said ve river boat animal',
 'nicholas newman crummles said mr smike tim noggs replied sir',
 'marius cosette thénardier father gillenormand man courfeyrac rue jondrette francs']